### <font color='purple'> Part 3 of the Titanic Case -- Feature Engineering</font>

Here comes to the most tricky part. As many Kagglers said, Kaggle competition is more like competing in feature engineering and in most cases, applying a right model isn't quite a big deal. Although it's only a startup project, the lessons we can learn in feature engineering is pretty typical.

#### 1. Title in the same field

Maybe people in some title tend to be saved. OMG, I'm just guessing......

In [2]:
import numpy as np
import pandas as pd

In [3]:
# Find a substring in a string; will be used when finding title
def substrings_in_string(big_string, substrings):
    for substring in substrings:
        if str.find(big_string, substring) != -1:
            return substring
    return np.nan

In [4]:
from sklearn import preprocessing

# Used for transform features later
# LabelEncoder will label distinct value but will not split into 0-1 columns like one-hot
le = preprocessing.LabelEncoder()
enc = preprocessing.OneHotEncoder()

# All the feature works in a function
def clean_data(df):
    #df.Fare = df.Fare.map(lambda x:np.nan if x==0)
    title_list=['Mrs', 'Mr', 'Master', 'Miss', 'Major', 'Rev',
                'Dr', 'Ms', 'Mlle','Col', 'Capt', 'Mme', 'Countess',
                'Don', 'Jonkheer']
    df['Title'] = df['Name'].map(lambda x:substrings_in_string(x,title_list))
    # Process speical titles into four kinds
    # Miss, Mrs, Mr, Master
    def replace_titles(x):
        title=x['Title']
        if title in ['Mr','Don', 'Major', 'Capt', 'Jonkheer', 'Rev', 'Col']:
            return 'Mr'
        elif title in ['Master']:
            return 'Master'
        elif title in ['Countess', 'Mme','Mrs']:
            return 'Mrs'
        elif title in ['Mlle', 'Ms','Miss']:
            return 'Miss'
        elif title =='Dr':
            if x['Sex']=='Male':
                return 'Mr'
            else:
                return 'Mrs'
        elif title =='':
            if x['Sex']=='Male':
                return 'Master'
            else:
                return 'Miss'
        else:
            return title
    df['Title'] = df.apply(replace_titles,axis=1) # every column means row by row
    
    # Define family attribute
    df['Family_Size'] = df['SibSp'] + df['Parch']
    df['Family'] = df['SibSp']*df['Parch']
    
    # Transform sex to int value
    df['Gender'] = df['Sex'].map({'female':0, 'male':1})
    
    # Fill null value in age with mean of same title
    df['AgeFill'] = df['Age']
    mean_ages = np.zeros(4)
    mean_ages[0] = np.average(df[df.Title == 'Miss']['Age'].dropna())
    mean_ages[1] = np.average(df[df.Title == 'Mrs']['Age'].dropna())
    mean_ages[2] = np.average(df[df.Title == 'Mr']['Age'].dropna())
    mean_ages[3] = np.average(df[df.Title == 'Master']['Age'].dropna())
    df.loc[(df.Title == 'Miss') & (df.Age.isnull()), 'AgeFill'] = mean_ages[0]
    df.loc[(df.Title == 'Mrs') & (df.Age.isnull()), 'AgeFill'] = mean_ages[1]
    df.loc[(df.Title == 'Mr') & (df.Age.isnull()), 'AgeFill'] = mean_ages[2]
    df.loc[(df.Title == 'Master') & (df.Age.isnull()), 'AgeFill'] = mean_ages[3]
    
    # Set category for different age ranges
    df['AgeCat'] = df['AgeFill']
    df.loc[(df.AgeFill <= 10), 'AgeCat'] = 'child'
    df.loc[(df.AgeFill > 60), 'AgeCat'] = 'aged'
    df.loc[(df.AgeFill > 10) & (df.AgeFill <= 30),'AgeCat'] = 'adult'
    df.loc[(df.AgeFill > 30) & (df.AgeFill <= 60),'AgeCat'] = 'senior'
    
    df.Embarked = df.Embarked.fillna('S')
    
    df.loc[ df.Cabin.isnull()==True,'Cabin'] = 0
    df.loc[ df.Cabin.isnull()==False,'Cabin'] = 1
    df.loc[(df.Fare.isnull()),'Fare'] = df.loc[(df.Pclass == 3),'Fare'].mean()
    # Calculate the fare by family size the larger the family size, the higher the fare
    df['Fare_Per_Person']=df['Fare']/(df['Family_Size']+1)
    
    # Combine features
    df['AgeClass']=df['AgeFill']*df['Pclass']
    df['ClassFare']=df['Pclass']*df['Fare_Per_Person']
    
    df['HighLow']='Low'
    #df.Fare_Per_Person[df.Fare_Per_Person.isnull()] = 
    df.loc[ (df.Fare_Per_Person<8),'HighLow'] = 'Low'
    df.loc[ (df.Fare_Per_Person>=8) ,'HighLow'] = 'High'
    print(df.HighLow.value_counts())
    
    dummies_Title = pd.get_dummies(df.Title, prefix='Title')
    dummies_AgeCat = pd.get_dummies(df.AgeCat, prefix='AgeCat')
    dummies_Embarked = pd.get_dummies(df.Embarked, prefix='Embarked')
    le.fit(df['HighLow'])
    x_hl = le.transform(df['HighLow'])
    df['HighLow'] = x_hl.astype(np.int32)
    
    df = pd.concat([df, dummies_Title, dummies_AgeCat,dummies_Embarked], axis=1)
    df.drop(['PassengerId', 'Name', 'Age', 'Ticket', 'Title', 'AgeCat', 'AgeFill', 'Embarked',
            'Sex','Fare'],axis=1,inplace=True)
    
    return df
    
    

In [5]:
train_data = pd.read_csv("train.csv")
train_df = clean_data(train_data)

High    504
Low     387
Name: HighLow, dtype: int64


In [6]:
train_df.head(10)

,Survived,Pclass,SibSp,Parch,Cabin,Family_Size,Family,Gender,Fare_Per_Person,AgeClass,...,Title_Miss,Title_Mr,Title_Mrs,AgeCat_adult,AgeCat_aged,AgeCat_child,AgeCat_senior,Embarked_C,Embarked_Q,Embarked_S
0,0,3,1,0,1,1,0,1,3.62500,66.000000,...,0,1,0,1,0,0,0,0,0,1
1,1,1,1,0,1,1,0,0,35.64165,38.000000,...,0,0,1,0,0,0,1,1,0,0
2,1,3,0,0,1,0,0,0,7.92500,78.000000,...,1,0,0,1,0,0,0,0,0,1
3,1,1,1,0,1,1,0,0,26.55000,35.000000,...,0,0,1,0,0,0,1,0,0,1
4,0,3,0,0,1,0,0,1,8.05000,105.000000,...,0,1,0,0,0,0,1,0,0,1
5,0,3,0,0,1,0,0,1,8.45830,98.675971,...,0,1,0,0,0,0,1,0,1,0
6,0,1,0,0,1,0,0,1,51.86250,54.000000,...,0,1,0,0,0,0,1,0,0,1
7,0,3,3,1,1,4,3,1,4.21500,6.000000,...,0,0,0,0,0,1,0,0,0,1
8,1,3,0,2,1,2,0,0,3.71110,81.000000,...,0,0,1,1,0,0,0,0,0,1
9,1,2,1,0,1,1,0,0,15.03540,28.000000,...,0,0,1,1,0,0,0,1,0,0


In [7]:
train_df.columns

Index(['Survived', 'Pclass', 'SibSp', 'Parch', 'Cabin', 'Family_Size',
       'Family', 'Gender', 'Fare_Per_Person', 'AgeClass', 'ClassFare',
       'HighLow', 'Title_Master', 'Title_Miss', 'Title_Mr', 'Title_Mrs',
       'AgeCat_adult', 'AgeCat_aged', 'AgeCat_child', 'AgeCat_senior',
       'Embarked_C', 'Embarked_Q', 'Embarked_S'],
      dtype='object')

In [8]:
from sklearn.cross_validation import train_test_split
filt_regex = "Survived|Pclass|SibSp|Parch|Cabin|Family|Family_Size|Gender|Fare_Per_Person|AgeClass|ClassFare|\
              HigLow|Title_.*|AgeCat_.*|Embarked_.*"
df_array = train_df.filter(regex=filt_regex).as_matrix()
y = list(df_array[:,0])
X = list(df_array[:,1:])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

/Users/captunez/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [10]:
# Report the best estimators(default: 3)
def report(grid_scores, n_top=3):
    top_scores = sorted(grid_scores, key=itemgetter(1), reverse=True)[:n_top]
    for i, score in enumerate(top_scores):
        print("Model with rank: {0}".format(i + 1))
        print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
              score.mean_validation_score,
              np.std(score.cv_validation_scores)))
        print("Parameters: {0}".format(score.parameters))
        print("")

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.metrics import classification_report
from operator import itemgetter
from sklearn.cross_validation import cross_val_score

clf=RandomForestClassifier(n_estimators=2000, criterion='entropy', max_depth=6, min_samples_split=2,
  min_samples_leaf=3, max_features='auto',bootstrap=True, oob_score=True, n_jobs=1, random_state=1,
  verbose=0)
param_grid = dict()
#param_grid = dict({'n_estimators':[200, 500, 1000, 2000], 
#                 'min_samples_leaf':[1,3,5], 'oob_score':[True, False]})
# pipeline=Pipeline([ ('clf',clf) ])  
# when using Pipeline, you can put multiple models and grid search the best sets of parameters
# Cross validation is used to compare the performance of taking different parameters in a model. Using 
# validation data instead of test data is to avoid test data leak into the model by adjusting the parameters
# best for the test data.
# There are kinds of cv method. Kfold: split without shuffle; Stratified: keep the percentage of classes
# avoid imbalanced distributions
grid_search = GridSearchCV(clf, param_grid=param_grid, verbose=3,scoring='accuracy',\
cv=StratifiedShuffleSplit(y_train, n_iter=5, test_size=0.2, train_size=None,  \
random_state=1)).fit(X_train, y_train)
print("Best score: %0.3f" % grid_search.best_score_)
print(grid_search.best_estimator_)
report(grid_search.grid_scores_)

print('-----grid search end------------')
print ('on all train set')
scores = cross_val_score(grid_search.best_estimator_, X_train, y_train,cv=3,scoring='accuracy')
print(scores.mean(),scores)
print ('on test set')
scores = cross_val_score(grid_search.best_estimator_, X_test, y_test,cv=3,scoring='accuracy')
print(scores.mean(),scores)

# Classification report 

print(classification_report(y_train, grid_search.best_estimator_.predict(X_train) ))
print('test data')
print(classification_report(y_test, grid_search.best_estimator_.predict(X_test) ))


/Users/captunez/anaconda/lib/python3.6/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV]  ................................................................
[CV] ....................................... , score=0.811189 -   3.0s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.0s remaining:    0.0s


[CV] ....................................... , score=0.818182 -   2.9s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    5.9s remaining:    0.0s


[CV] ....................................... , score=0.867133 -   2.7s
[CV]  ................................................................
[CV] ....................................... , score=0.846154 -   2.6s
[CV]  ................................................................
[CV] ....................................... , score=0.881119 -   2.7s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   13.8s finished


Best score: 0.845
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=6, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=3,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=2000, n_jobs=1, oob_score=True, random_state=1,
            verbose=0, warm_start=False)
Model with rank: 1
Mean validation score: 0.845 (std: 0.027)
Parameters: {}

-----grid search end------------
on all train set
0.832894174619 [ 0.80672269  0.8487395   0.84322034]
on test set
0.75965545985 [ 0.7704918   0.72881356  0.77966102]
             precision    recall  f1-score   support

          0       0.86      0.94      0.90       443
          1       0.89      0.75      0.82       269

avg / total       0.87      0.87      0.87       712

test data
             precision    recall  f1-score   support

          0       0.77      0.91      0.83       106
          1       0.81   

In [171]:
test_data = pd.read_csv('test.csv')
test_df = clean_data(test_data)

High    234
Low     184
Name: HighLow, dtype: int64


In [172]:
test_df.head()

,Pclass,SibSp,Parch,Cabin,Family_Size,Family,Gender,Fare_Per_Person,AgeClass,ClassFare,...,Title_Miss,Title_Mr,Title_Mrs,AgeCat_adult,AgeCat_aged,AgeCat_child,AgeCat_senior,Embarked_C,Embarked_Q,Embarked_S
0,3,0,0,1,0,0,1,7.829200,103.5,23.4876,...,0,1,0,0,0,0,1,0,1,0
1,3,1,0,1,1,0,0,3.500000,141.0,10.5000,...,0,0,1,0,0,0,1,0,0,1
2,2,0,0,1,0,0,1,9.687500,124.0,19.3750,...,0,1,0,0,1,0,0,0,1,0
3,3,0,0,1,0,0,1,8.662500,81.0,25.9875,...,0,1,0,1,0,0,0,0,0,1
4,3,1,1,1,2,1,0,4.095833,66.0,12.2875,...,0,0,1,1,0,0,0,0,0,1


In [173]:
test_array = test_df.filter(regex=filt_regex).as_matrix()
survived_pred = grid_search.best_estimator_.predict(test_array)


In [174]:
result_df = pd.DataFrame({'PassengerId':test_data.PassengerId, 'Survived':survived_pred})
result_df.to_csv("logistic_regression_predictions3.csv", index=False)

### <font color="purple"> Using Bagging to train a new model </font>

In [24]:
from sklearn.ensemble import BaggingClassifier
from sklearn import linear_model


In [25]:
clf = linear_model.LogisticRegression(C=1.0, penalty='l1',tol=1e-6)
# Bagging: using a base model and take random subsets of data
bagging_clf = BaggingClassifier(clf, n_estimators=10, max_samples=0.8, max_features=1.0, bootstrap=True, 
                              bootstrap_features=False, n_jobs=-1)

bagging_clf.fit(X_train, y_train)
print(classification_report(y_train, bagging_clf.predict(X_train) ))
print('test data')
print(classification_report(y_test, bagging_clf.predict(X_test) ))

             precision    recall  f1-score   support

          0       0.86      0.89      0.87       443
          1       0.81      0.75      0.78       269

avg / total       0.84      0.84      0.84       712

test data
             precision    recall  f1-score   support

          0       0.80      0.89      0.84       106
          1       0.80      0.67      0.73        73

avg / total       0.80      0.80      0.80       179



In [28]:
bagging_clf.get_params().keys()

dict_keys(['base_estimator__C', 'base_estimator__class_weight', 'base_estimator__dual', 'base_estimator__fit_intercept', 'base_estimator__intercept_scaling', 'base_estimator__max_iter', 'base_estimator__multi_class', 'base_estimator__n_jobs', 'base_estimator__penalty', 'base_estimator__random_state', 'base_estimator__solver', 'base_estimator__tol', 'base_estimator__verbose', 'base_estimator__warm_start', 'base_estimator', 'bootstrap', 'bootstrap_features', 'max_features', 'max_samples', 'n_estimators', 'n_jobs', 'oob_score', 'random_state', 'verbose', 'warm_start'])

In [33]:
clf = linear_model.LogisticRegression(C=1.0, penalty='l1',tol=1e-6)
# Bagging: using a base model and take random subsets of data
bagging_clf = BaggingClassifier(clf, n_estimators=30, max_samples=0.9, max_features=0.75, bootstrap=True, 
                              bootstrap_features=True, n_jobs=-1)

#param_grid = {'max_samples':[0.5, 0.7, 0.9], 'max_features':[0.5, 0.75, 1.0]}
#param_grid = {'bootstrap':[True, False], 'bootstrap_features':[True, False]}
param_grid = {}
grid_search = GridSearchCV(bagging_clf, param_grid=param_grid, verbose=3,scoring='accuracy',\
cv=StratifiedShuffleSplit(y_train, n_iter=5, test_size=0.2, train_size=None,  \
random_state=1)).fit(X_train, y_train)
print("Best score: %0.3f" % grid_search.best_score_)
print(grid_search.best_estimator_)
report(grid_search.grid_scores_)

print('-----grid search end------------')
print ('on all train set')
scores = cross_val_score(grid_search.best_estimator_, X_train, y_train,cv=3,scoring='accuracy')
print(scores.mean(),scores)
print ('on test set')
scores = cross_val_score(grid_search.best_estimator_, X_test, y_test,cv=3,scoring='accuracy')
print(scores.mean(),scores)

print(classification_report(y_train, grid_search.best_estimator_.predict(X_train) ))
print('test data')
print(classification_report(y_test, grid_search.best_estimator_.predict(X_test) ))

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV]  ................................................................
[CV] ....................................... , score=0.832168 -   0.4s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s


[CV] ....................................... , score=0.804196 -   0.4s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.8s remaining:    0.0s


[CV] ....................................... , score=0.839161 -   0.3s
[CV]  ................................................................
[CV] ....................................... , score=0.818182 -   0.3s
[CV]  ................................................................
[CV] ....................................... , score=0.860140 -   0.4s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    1.9s finished


Best score: 0.831
BaggingClassifier(base_estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=1e-06,
          verbose=0, warm_start=False),
         bootstrap=True, bootstrap_features=True, max_features=0.75,
         max_samples=0.9, n_estimators=30, n_jobs=-1, oob_score=False,
         random_state=None, verbose=0, warm_start=False)
Model with rank: 1
Mean validation score: 0.831 (std: 0.019)
Parameters: {}

-----grid search end------------
on all train set
0.825855766035 [ 0.79831933  0.8487395   0.83050847]
on test set
0.776975085672 [ 0.73770492  0.76271186  0.83050847]
             precision    recall  f1-score   support

          0       0.85      0.88      0.87       443
          1       0.79      0.75      0.77       269

avg / total       0.83      0.83      0.83       712

test data
             prec

In [35]:
test_data = pd.read_csv('test.csv')
test_df = clean_data(test_data)
test_array = test_df.filter(regex=filt_regex).as_matrix()
survived_pred = grid_search.best_estimator_.predict(test_array)
result_df = pd.DataFrame({'PassengerId':test_data.PassengerId, 'Survived':survived_pred})
result_df.to_csv("logistic_regression_predictions4.csv", index=False)

High    234
Low     184
Name: HighLow, dtype: int64
